In [ ]:
from init_notebook import *

In [ ]:
from experiments.datasets import PixelartDataset

ds = PixelartDataset()

In [ ]:
wang_patches = []
for wang_tiles in (wangtiles.WangTiles2E(), wangtiles.WangTiles2C()):
    wang_template = wang_tiles.create_template((32, 32), fade=3., padding=-.5)
    # VF.to_pil_image(wang_template.image)
    wang_patches.extend([
        i.unsqueeze(0) for i in iter_image_patches(wang_template.image, (32, 32))
    ])

wang_patches = torch.concat(wang_patches)
display(VF.to_pil_image(make_grid(wang_patches, nrow=4)))

wang_weights = wang_patches.flatten(1) / torch.norm(wang_patches.flatten(1), dim=1, keepdim=True)

In [ ]:
def norm_image_batch(input):
    return (input.flatten(1) / torch.norm(input.flatten(1), dim=1, keepdim=True)).reshape(input.shape)

def get_similarity(input, weights=wang_weights):
    input = input.flatten(1)
    input = input / torch.norm(input, dim=1, keepdim=True)
    return input @ weights.T

In [ ]:
patches = []
label_filter = PixelartDataset.LABELS.index("wall") 
for patch, label in DataLoader(ds, shuffle=True):
    if label == label_filter:
        patches.append(patch)
    if len(patches) >= 512:
        break
patches = torch.concat(patches)       
#patches = ds.sample(256)[0]
print(patches.shape)

In [ ]:
sim = get_similarity(patches)
print(wang_weights.shape)
px.imshow(sim.T, height=400, aspect=False)

In [ ]:
sim_t = sim.T
indices = sim_t.argsort(dim=1, descending=True)
grid = []
grid_labels = [""] * wang_patches.shape[0]
grid.extend(wang_patches)
for j in range(20):
    for i in range(wang_patches.shape[0]):
        grid.append(patches[indices[i][j]])
        grid_labels.append(round(float(sim_t[i, indices[i][j]]), 2))

VF.to_pil_image(make_grid_labeled(grid, labels=grid_labels, nrow=wang_patches.shape[0]))
#indices.shape    

# compare PCA

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(32)
pca.fit(ds.sample(len(ds))[0].flatten(1))
VF.to_pil_image(make_grid(torch.Tensor(pca.components_).reshape(-1, 4, 32, 32), normalize=True))

In [ ]:
pca_weights = torch.Tensor(pca.components_)
pca_weights /= torch.norm(pca_weights, dim=1, keepdim=True)

In [ ]:
sim_t = get_similarity(patches, pca_weights).T
print(sim_t.shape)
px.imshow(sim_t, height=400, aspect=False)

In [ ]:
indices = sim_t.argsort(dim=1, descending=True)
grid = []
grid_labels = [""] * wang_patches.shape[0]
grid.extend(pca_weights.reshape(-1, 3, 32, 32))
for j in range(20):
    for i in range(sim_t.shape[0]):
        grid.append(patches[indices[i][j]])
        grid_labels.append(round(float(sim_t[i, indices[i][j]]), 2))

VF.to_pil_image(make_grid_labeled(grid, labels=grid_labels, nrow=wang_patches.shape[0]))
#indices.shape    